In [132]:
import numpy as np
import h5py
import torch
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

# Helper Clases / Functions

In [133]:
def checkAccuracy(model, dataloader):
    y_pred_tot = []
    y_sample_tot = []
    for i_batch, sample_batched in enumerate(dataloader):
        X_sample, y_sample = sample_batched
        X_sample, y_sample = Variable(X_sample), Variable(y_sample)
        scores = model(X_sample)
        #loss = loss_fn(scores, y_sample.long())
        #print('val loss: %f' % loss)
        _, y_pred = torch.max(scores, 1)
        y_pred_tot.append(y_pred.data.numpy())
        y_sample_tot.append(y_sample.data.numpy())
    acc = np.mean(np.concatenate(y_pred_tot) == np.concatenate(y_sample_tot))
    return acc
class Flatten(nn.Module):
    def forward(self, x):
        N, E, T = x.size() 
        return x.view(N, -1)

# Load Data

In [134]:
A01T = h5py.File('./../project_datasets/A01T_slice.mat', 'r')
print (A01T)
X = np.copy(A01T['image'])
y = np.copy(A01T['type'])
X = X[:, 0:23, :] # only goes to 23 since you dont use last 3 channels
# X = np.moveaxis(X, -1, 0)
y = y[0,0:X.shape[0]:1]
#y = np.asarray(y, dtype=np.int32)
#subtract 769 so classes start at 0
y = y - 769
N, E, T = np.shape(X)
print (np.shape(X))

<HDF5 file "A01T_slice.mat" (mode r)>
(288, 23, 1000)


# Make DataLoaders

In [135]:
bs = 50
data = data_utils.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dset = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [N-100, 50, 50])
dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=bs, shuffle=True, num_workers=1) for x in ['train', 'val', 'test']}

# Define Model

In [136]:
dtype = torch.FloatTensor
model = nn.Sequential(
    Flatten(),
    nn.Linear(E*T, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, 4),
)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training

In [137]:
num_epochs = 10
for epoch in range(num_epochs):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        X_train, y_train = Variable(X_train), Variable(y_train)
        X_train.type(dtype)
        y_train.type(dtype)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(X_train)
        loss = loss_fn(outputs, y_train.long())        
        print('(%d batch) loss: %f' % (i, loss))
        loss.backward()
        optimizer.step()
    train_acc = checkAccuracy(model, dataloaders['train'])
    val_acc = checkAccuracy(model, dataloaders['val'])
    print('(Epoch %d / %d) train_acc: %f; val_acc: %f' % (epoch+1, num_epochs, train_acc, val_acc))

(0 batch) loss: 2.427359
(1 batch) loss: 81.606323
(2 batch) loss: 62.795219
(3 batch) loss: 88.862206
(Epoch 1 / 10) train_acc: 0.739362; val_acc: 0.380000
(0 batch) loss: 13.012598
(1 batch) loss: 14.709541
(2 batch) loss: 16.217756
(3 batch) loss: 9.572400
(Epoch 2 / 10) train_acc: 0.872340; val_acc: 0.300000
(0 batch) loss: 3.995305
(1 batch) loss: 1.166429
(2 batch) loss: 3.172714
(3 batch) loss: 1.897641
(Epoch 3 / 10) train_acc: 0.968085; val_acc: 0.320000
(0 batch) loss: 0.296557
(1 batch) loss: 0.064464
(2 batch) loss: 0.366129
(3 batch) loss: 0.051982
(Epoch 4 / 10) train_acc: 0.994681; val_acc: 0.320000
(0 batch) loss: 0.004595
(1 batch) loss: 0.044119
(2 batch) loss: 0.002828
(3 batch) loss: 0.121900
(Epoch 5 / 10) train_acc: 1.000000; val_acc: 0.340000
(0 batch) loss: 0.000714
(1 batch) loss: 0.037041
(2 batch) loss: 0.008389
(3 batch) loss: 0.450589
(Epoch 6 / 10) train_acc: 0.994681; val_acc: 0.340000
(0 batch) loss: 0.106901
(1 batch) loss: 0.001490
(2 batch) loss: 0.03